In [ ]:
#!pip install mysqlclient
#!pip install bellmanford
#!pip install networkx
#!conda install -c anaconda networkx 

In [ ]:
from tqdm import tqdm
import MySQLdb as MySQLdb
import ast
import numpy as np
from ArbitrageGraph import ArbitrageGraph


In [ ]:
arbitrageGraph = ArbitrageGraph()

In [ ]:
sql="SELECT exchange, pair, bids, asks, id FROM orderbook WHERE exchange IN ('coinfloor','kraken','bitfinex','bittrex','gdax','bitstamp','coinbase','poloniex') ORDER BY ID LIMIT 2000;"

In [ ]:
db = MySQLdb.connect(
    host="orderbook.cyifbgm0zwt0.eu-west-2.rds.amazonaws.com",
    user="admin",
    passwd="123Qwe123Qwe",
    db="orderbook",
    port=3306)
cursor = db.cursor()
nof_rows=cursor.execute(sql)

In [ ]:
#for row in tqdm(cursor.fetchall()):
for row in tqdm(cursor.fetchone()):
    exchangename = row[0]
    symbol = row[1]
    bids_str = row[2]
    asks_str = row[3]
    try:
        l_ask = ast.literal_eval(asks_str)[0][0]
        h_bid = ast.literal_eval(bids_str)[0][0]
        #print("%s,%s,%f,%f" % (exchangename, symbol, h_bid, l_ask))
        length, nodes, negative_cycle = arbitrageGraph.update_point(
            symbol,
            exchangename,
            0.02,
            l_ask,
            h_bid)
        
        if negative_cycle == True:
            edges=arbitrageGraph.nodeslist_to_edges(nodes)
            print("length:",length,"ratio",np.exp(-length),'nodes',nodes,'edges',edges)
            #arbitrageGraph.plot_graph()
    except IndexError:
        print("*** Invalid orderbook ***")


In [ ]:
db.close()